## Basing the implementation on the one performed by Pier

In [3]:
import os
import json
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import pandas as pd
import gensim 

df_ori = pd.read_csv('../data/data job posts.csv')
df_ori.head()
print(df_ori.shape)
df = df_ori.drop_duplicates(['RequiredQual'])
print(df.shape)
print("Removed {0} duplicates (based on RequiredQual)".format(df_ori.shape[0]-df.shape[0]))

print(df["RequiredQual"].head())

df["RequiredQual"] = df["RequiredQual"].astype(str)

df['RequiredQual_processed'] = df.RequiredQual.apply(gensim.utils.simple_preprocess)
df = df.drop(["RequiredQual", "jobpost"], axis=1)
print(df.head())

(19001, 24)
(16689, 24)
Removed 2312 duplicates (based on RequiredQual)
0    To perform this job successfully, an\nindividu...
1    - Bachelor's Degree; Master's is preferred;\n-...
2    - Degree in environmentally related field, or ...
3    - Advanced degree in public health, social sci...
4    - University degree; economical background is ...
Name: RequiredQual, dtype: object


G:\anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


           date                                              Title  \
0   Jan 5, 2004                            Chief Financial Officer   
1   Jan 7, 2004  Full-time Community Connections Intern (paid i...   
2   Jan 7, 2004                                Country Coordinator   
3   Jan 7, 2004                                     BCC Specialist   
4  Jan 10, 2004                                 Software Developer   

                                           Company AnnouncementCode Term  \
0             AMERIA Investment Consulting Company              NaN  NaN   
1  International Research & Exchanges Board (IREX)              NaN  NaN   
2        Caucasus Environmental NGO Network (CENN)              NaN  NaN   
3                                     Manoff Group              NaN  NaN   
4                           Yerevan Brandy Company              NaN  NaN   

  Eligibility Audience StartDate                             Duration  \
0         NaN      NaN       NaN                 

G:\anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Train the word 2 vec model on the required qualifications after basic pre-processing [refer](http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W7l-lWgzbIU)

In [6]:
#word2vec_model = gensim.models.Word2Vec(df['RequiredQual_processed'], size=100, window=5, min_count=1, workers=4)
#word2vec_model.save("../outputs/required_qual.model")

In [10]:
required_qual_model = gensim.models.Word2Vec.load("../outputs/required_qual.model")
required_qual_model.wv['finance'].shape

(100,)

## Get the average word 2 vec vectors for the documents

In [15]:
import numpy as np

def get_average_word2vec_for_qual(token_list):
    if token_list:
        num_tokens = len(token_list)
        word_2_vec_sum = np.zeros(100)
        for token in token_list:
            word_2_vec_sum += required_qual_model.wv[token]
        
        #print("SUM: " + str(word_2_vec_sum))
        #print("LEN: " + str(num_tokens))
        average_word_2_vec = word_2_vec_sum/num_tokens
        #print("AVG" + str(average_word_2_vec))
        return average_word_2_vec
    return None


In [16]:
get_average_word2vec_for_qual(df['RequiredQual_processed'][0])

array([ 0.10884372, -0.39476334, -0.07866897, -0.03640246, -0.42963778,
       -1.05963732, -0.10694249, -0.6946278 , -0.23645025, -0.40056211,
        0.34469219,  0.35970644,  0.31005736, -0.26279422, -0.16430086,
       -0.04659212,  0.33829459,  0.42903162, -0.07504804,  0.07977514,
        0.13282309, -0.18764175,  0.01966968,  0.33593797, -0.57092042,
       -0.21030081,  0.10690218, -0.20199957,  0.54535492, -0.8270151 ,
       -0.0185453 , -0.11861162, -0.14917046, -0.02694462, -0.21019031,
        0.45789709, -0.18619073, -0.44579528, -0.05107523,  0.02861218,
       -0.43856222, -0.15693517, -0.019054  ,  0.21764722,  0.12828929,
       -0.0537546 , -0.30590729, -0.15455936,  0.04036621, -0.02763412,
        0.19034366, -0.38321557, -0.34897171,  0.14813457, -0.33465318,
       -0.12326898,  0.46683668,  0.04076201,  0.46264456, -0.31586635,
       -0.13638585, -0.02607759,  0.0975218 ,  0.3020699 , -0.09764642,
       -0.18479594, -0.73101058,  0.35582876,  0.10177838, -0.17

In [17]:
df["RequiredQual_Avg_Word2Vec"] = df['RequiredQual_processed'].apply(get_average_word2vec_for_qual)
df["RequiredQual_Avg_Word2Vec"].head()


0    [0.10884372287936503, -0.3947633358119212, -0....
1    [-0.05059807275732358, -0.3633127188723948, 0....
2    [-0.06786293610930443, -0.16080295515379736, -...
3    [0.031613347048146855, -0.3168140259726594, -0...
4    [0.14961214930835096, -0.47536581445654685, -0...
Name: RequiredQual_Avg_Word2Vec, dtype: object

In [73]:
num_clusters = 3

In [74]:
num_rows = df["RequiredQual_Avg_Word2Vec"].shape[0]
num_rows

16689

In [75]:
word_2_vec_values = df["RequiredQual_Avg_Word2Vec"].values
print(word_2_vec_values.shape)
word_2_vec_values[0]
word_2_vec_values = np.array([np.array(x) for x in word_2_vec_values])
print(word_2_vec_values.shape)

(16689,)
(16689, 100)


In [76]:
word_2_vec_values[0]

array([ 0.10884372, -0.39476334, -0.07866897, -0.03640246, -0.42963778,
       -1.05963732, -0.10694249, -0.6946278 , -0.23645025, -0.40056211,
        0.34469219,  0.35970644,  0.31005736, -0.26279422, -0.16430086,
       -0.04659212,  0.33829459,  0.42903162, -0.07504804,  0.07977514,
        0.13282309, -0.18764175,  0.01966968,  0.33593797, -0.57092042,
       -0.21030081,  0.10690218, -0.20199957,  0.54535492, -0.8270151 ,
       -0.0185453 , -0.11861162, -0.14917046, -0.02694462, -0.21019031,
        0.45789709, -0.18619073, -0.44579528, -0.05107523,  0.02861218,
       -0.43856222, -0.15693517, -0.019054  ,  0.21764722,  0.12828929,
       -0.0537546 , -0.30590729, -0.15455936,  0.04036621, -0.02763412,
        0.19034366, -0.38321557, -0.34897171,  0.14813457, -0.33465318,
       -0.12326898,  0.46683668,  0.04076201,  0.46264456, -0.31586635,
       -0.13638585, -0.02607759,  0.0975218 ,  0.3020699 , -0.09764642,
       -0.18479594, -0.73101058,  0.35582876,  0.10177838, -0.17

In [77]:
from sklearn.cluster import KMeans
from sklearn import metrics

kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++')
idx = kmeans_clustering.fit_predict(word_2_vec_values)
print(idx.shape)

(16689,)


In [78]:
idx[:10]

array([2, 1, 0, 2, 2, 1, 1, 2, 0, 2])

In [79]:
df["RequiredQual_Cluster_Index"] = idx
df["RequiredQual_Cluster_Index"].head()

0    2
1    1
2    0
3    2
4    2
Name: RequiredQual_Cluster_Index, dtype: int32

In [80]:
# Special processing to remove the unwanted terms
def preprocess(tokens):
    tokens_nop = [t for t in tokens if t not in string.punctuation]
    tokens_nop = [t.lower() for t in tokens_nop]
    wnl = nltk.WordNetLemmatizer()
    stop = stopwords.words('english')
    tokens_nostop = [t for t in tokens_nop if t not in stop]
    tokens_lem = [wnl.lemmatize(t) for t in tokens_nostop]
    tokens_clean = [t for t in tokens_lem if len(t) >= 3]  # simple way to remove the offending " punctuations
    return tokens_clean


df['RequiredQual_processed'] = df["RequiredQual_processed"].apply(preprocess)

In [81]:
print("Silhouette Coefficient for clusters: %0.3f"
          % metrics.silhouette_score(word_2_vec_values, kmeans_clustering.labels_))

Silhouette Coefficient for clusters: 0.156


In [82]:
grouped_df = df.groupby("RequiredQual_Cluster_Index")["RequiredQual_processed"].sum()
grouped_df

RequiredQual_Cluster_Index
0    [degree, environmentally, related, field, year...
1    [bachelor, degree, master, preferred, excellen...
2    [perform, job, successfully, individual, must,...
Name: RequiredQual_processed, dtype: object

In [83]:
grouped_df.shape

(3,)

In [84]:
len(grouped_df[0])

166343

In [85]:
len(df["RequiredQual_processed"][0])

249

In [87]:
from collections import Counter
for idx, g in enumerate(grouped_df):
    c = Counter(g)
    print("Cluster # {0}".format(idx))
    tokens = [x[0] for x in c.most_common(50)]
    print(" ".join(tokens))

Cluster # 0
knowledge experience skill language english year work armenian excellent good russian degree least field accounting computer education university communication higher plus office ability relevant finance economics management related banking strong professional fluency software team financial legislation international excel bank written working business preferably law word analytical system technical preferable advanced
Cluster # 1
skill ability knowledge work experience language excellent english communication armenian good russian strong year degree team computer management least field written office university pressure education personality plus organizational interpersonal higher working analytical high business relevant fluency excel related word responsibility problem time accounting sense detail project self economics oriented oral
Cluster # 2
experience knowledge ability skill work year language good development english degree least strong excellent communication man

## Observations from the clusters
a. Language skill is required in almost all the job positions - esp. Armenian, Russian and English
b. Cluster 0 is more on the finance, 1 (?) and 2 is mostly IT related